## Techniche - Topic Model

In [ ]:
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize

import json
import requests
import re

from topic_model import tokenize_docs

import matplotlib.pyplot as plt
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
from pprint import pprint

In [2]:
np.random.seed(3)

In [3]:
# uncomment to download stop words from nltk
# nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### import data from PatentsView API

In [4]:
# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 50
pd.set_option('display.max_rows', 50)

# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

# build query
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
resp = requests.get(endpoint_url, params=params)
results = resp.json()

#### structure data

In [5]:
# extract metadata from response
print("status code:", resp.status_code,';', "reason:", resp.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data = results['patents']
# data[0]
df = pd.DataFrame(data)
df.head(3)

# df.columns

df = df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id', 
         'patent_year', 
         'patent_type', 
         'patent_kind']]

df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

df.patent_firstnamed_assignee_id.value_counts()[:10]

# list of assignees with > 20 patents in df dataset
assignees_list = ['org_q9Bn28RHhpYrQjKvraAH', 'org_JZguWDMfFOBX2wBI9pnD', 'org_ID497r4tFbCIaMBjGAST', 
                  'org_rDyHZBYWMcBEtnkHt05L', 'org_p6ofWD2xFNSnyYkj6wpA', 'org_EilEWQcC6UiqHcSGx9mb',
                  'org_ccMMcUijAIsKIxUqMTyP', 'org_Vbc6obpnxWM42d0HjlXY', 'org_9D8x1qL3IRASp6GG7Glu',
                  'org_2wAdIFKssfcLHpZq0u4H', 'org_iwO2oOJ6VIBd9fAuP7G6', 'org_70D1lR89kQnFiCFdJ6s5',
                  'org_vojVnDkT9CamDETqbqJC']

df_20pats = df[df['patent_firstnamed_assignee_id'].isin(assignees_list) ]

df_20pats.sort_values(by=['patent_date'], inplace=True)

df_20pats[['patent_number','patent_title_abstract', 'patent_firstnamed_assignee_id']].head(1)

status code: 200 ; reason: OK
total_patent_count: 2482 ; patents_per_page: 2482


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,patent_number,patent_title_abstract,patent_firstnamed_assignee_id
2479,4502128,Translation between natural languages An input...,org_70D1lR89kQnFiCFdJ6s5


#### Partition data into train and test sets

In [6]:
train_20pats = df_20pats[:894]
len(train_20pats)

test_20pats = df_20pats[894:]
len(test_20pats)

224

In [7]:
len(train_20pats)

894

In [8]:
data = train_20pats.patent_title_abstract.values.tolist()
len(data)

894

In [9]:
data

['Translation between natural languages An input sentence described by a first natural language is sectioned into individual words. Parts of speech corresponding to the individual words are retrieved from a lexical word storage, whereby the input sentence is described by a corresponding string of the parts-of-speech as retrieved. A translation pattern table is previously prepared which defines correspondence between patterns of strings of parts-of-speech for the first natural language and those for a second natural language by which corresponding output sentence is to be described. By referring to the translation pattern table, the string of the parts-of-speech of the input sentence is transformed into a corresponding string of the parts-of-speech for the second natural language. The output sentence described by the second natural language is generated by sequencing target words in accordance with the sequential order of the parts of speech of the string pattern obtained after the tran

In [14]:
def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

In [21]:
tokenized_docs = tokenize_docs(data)

In [22]:
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [24]:
data = clean_docs(tokenized_docs)

In [46]:
stop_words = stopwords.words('english')

In [47]:
def remove_stopwords(clean_docs):
    filtered_docs = []
    for doc in clean_docs:
       filtered_docs.append([[word for word in clean_docs if word not in stop_words]])
    return filtered_docs

In [ ]:
remove_stopwords(data)

In [34]:
# train bigram phrases model
bigram_model = Phrases(data, min_count=1, threshold=1)

In [35]:
# train trigram phrases model
trigram_model = Phrases(bigram_model[data], threshold=100)  

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [37]:
bigram_model[data[0]]

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['Translation',
 'between',
 'natural_languages',
 'An_input',
 'sentence_described',
 'by_a',
 'first_natural',
 'language',
 'is',
 'sectioned',
 'into_individual',
 'words',
 'Parts',
 'of',
 'speech',
 'corresponding_to',
 'the',
 'individual_words',
 'are_retrieved',
 'from_a',
 'lexical',
 'word',
 'storage',
 'whereby',
 'the_input',
 'sentence_is',
 'described_by',
 'a',
 'corresponding_string',
 'of_the',
 'as',
 'retrieved_A',
 'translation_pattern',
 'table_is',
 'previously',
 'prepared',
 'which_defines',
 'correspondence',
 'between',
 'patterns',
 'of',
 'strings',
 'of',
 'for',
 'the_first',
 'natural_language',
 'and',
 'those_for',
 'a_second',
 'natural_language',
 'by',
 'which',
 'corresponding',
 'output_sentence',
 'is',
 'to_be',
 'described',
 'By',
 'referring_to',
 'the',
 'translation_pattern',
 'table',
 'the',
 'string_of',
 'the',
 'of_the',
 'input_sentence',
 'is_transformed',
 'into_a',
 'corresponding_string',
 'of_the',
 'for',
 'the_second',
 'natu

In [42]:
trigram_model[bigram_model[data[0]]]

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['Translation',
 'between',
 'natural_languages',
 'An_input',
 'sentence_described',
 'by_a',
 'first_natural',
 'language',
 'is',
 'sectioned',
 'into_individual',
 'words',
 'Parts',
 'of',
 'speech',
 'corresponding_to',
 'the',
 'individual_words',
 'are_retrieved',
 'from_a',
 'lexical',
 'word',
 'storage',
 'whereby',
 'the_input',
 'sentence_is',
 'described_by',
 'a',
 'corresponding_string',
 'of_the',
 'as',
 'retrieved_A',
 'translation_pattern',
 'table_is',
 'previously',
 'prepared',
 'which_defines',
 'correspondence',
 'between',
 'patterns',
 'of',
 'strings',
 'of',
 'for',
 'the_first',
 'natural_language',
 'and',
 'those_for',
 'a_second',
 'natural_language',
 'by',
 'which',
 'corresponding',
 'output_sentence',
 'is',
 'to_be',
 'described',
 'By',
 'referring_to',
 'the',
 'translation_pattern',
 'table',
 'the',
 'string_of',
 'the',
 'of_the',
 'input_sentence',
 'is_transformed',
 'into_a',
 'corresponding_string',
 'of_the',
 'for',
 'the_second',
 'natu

In [ ]:
# create bigram model
bigram = gensim.models.Phrases(train_20pats.patent_title_abstract, min_count=5, threshold=100)

In [30]:
# create trigram
trigram = gensim.models.Phrases(bigram_phrases[bigram_phrases], threshold=100)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


TypeError: 'int' object is not iterable

In [ ]:
# initiatialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram)
trigram_model = gensim.models.phrases.Phraser(trigram)

In [ ]:
print(bigram_model[train_20pats.patent_title_abstract[0]])

In [ ]:
print(trigram_mod[bigram_mod[train_20pats.patent_title_abstract[0]]])

In [ ]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])